# Domain-Specific Medical LLM Q&A Agent (RAG + LLM)

**A Retrieval-Augmented Generation System for Safe, Evidence-Based Medical Question Answering**

---

## 📋 Project Overview

This notebook implements a **Domain-Specific Medical Q&A Agent** using **Retrieval-Augmented Generation (RAG)** combined with a powerful **LLM**. The system is designed to answer medical questions using **only trusted, publicly available medical guidelines** (e.g., WHO, CDC), making it suitable for health-tech applications.

### Key Features:
- ✅ **RAG Architecture**: Combines retrieval with LLM generation for accurate, evidence-based answers
- ✅ **Medical Domain Focus**: Uses only trusted medical guidelines (WHO, CDC)
- ✅ **Safety First**: Built-in safety checks and medical disclaimers
- ✅ **Citation Support**: All answers include source citations
- ✅ **Production-Ready**: Modular, well-documented, and deployment-ready

---

## 🏗️ Architecture Overview

### What is RAG?

**Retrieval-Augmented Generation (RAG)** is a technique that enhances LLM responses by:

1. **Retrieving** relevant documents from a knowledge base
2. **Augmenting** the LLM prompt with retrieved context
3. **Generating** answers based on the provided context

### Why RAG for Medical Applications?

- **Reduces Hallucinations**: LLMs alone can generate plausible but incorrect medical information
- **Evidence-Based**: Answers are grounded in actual medical guidelines
- **Up-to-Date**: Knowledge base can be updated without retraining the model
- **Transparency**: Citations show where information comes from
- **Safety**: Built-in validation prevents harmful responses

### Pipeline Flow:

```
Medical Documents → Chunking → Embeddings → Vector Store
                                                     ↓
User Query → Embedding → Vector Search → Retrieve Top-K Docs
                                                     ↓
Retrieved Context + Query → LLM Prompt → Generated Answer + Citations
```

---

## ⚠️ Medical Disclaimer

**IMPORTANT**: This system provides general medical information for educational purposes only. It is **NOT** a substitute for professional medical advice, diagnosis, or treatment. Always consult qualified healthcare providers for personal medical concerns.


## 🔧 Section 1: Setup & Installation

### Google Colab Setup

If running in Google Colab, enable GPU for faster processing:
- Runtime → Change runtime type → GPU (T4 or better)

### Install Dependencies


In [ ]:
# Install required packages
%pip install -q torch transformers sentence-transformers faiss-cpu accelerate numpy pandas pyyaml requests tqdm


In [ ]:
# Check GPU availability
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("Using CPU (slower but works)")


In [ ]:
# Import all required libraries
import sys
from pathlib import Path
import json
import yaml
from typing import List, Dict, Optional
import numpy as np
import pandas as pd
from tqdm import tqdm

# Add utils to path
sys.path.append(str(Path.cwd()))

# Import utility modules
from utils import (
    load_data, clean_text, structure_documents,
    MedicalChunker, extract_metadata,
    EmbeddingGenerator,
    VectorStore, build_vector_store,
    RetrievalEngine,
    RAGPipeline,
    RAGEvaluator,
    MedicalSafetyChecker
)

print("✅ All imports successful!")


## 📥 Section 2: Data Ingestion

### Why Preprocessing Matters for Medical Accuracy

Medical documents often contain:
- Formatting inconsistencies
- Special characters and symbols
- Multiple languages or translations
- Structured sections (symptoms, treatment, prevention)

Proper preprocessing ensures:
- **Better Retrieval**: Clean text improves embedding quality
- **Reduced Noise**: Removes irrelevant formatting
- **Consistent Structure**: Makes chunking more effective
- **Metadata Extraction**: Identifies diseases, sections, sources


In [ ]:
# Load configuration
config_path = Path("config/config.yaml")
if config_path.exists():
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
else:
    # Default configuration
    config = {
        'chunking': {'chunk_size': 1000, 'chunk_overlap': 200},
        'embedding': {'model_name': 'sentence-transformers/all-MiniLM-L6-v2', 'batch_size': 32},
        'retrieval': {'top_k': 5},
        'llm': {'model_name': 'google/flan-t5-base', 'max_length': 512}
    }

print("Configuration loaded:")
print(json.dumps(config, indent=2))


In [ ]:
# Load medical documents
data_file = Path("data/sample_medical_guidelines.txt")

if data_file.exists():
    raw_documents = load_data(data_file, source_type="file")
    print(f"✅ Loaded {len(raw_documents)} document(s)")
    print(f"Total characters: {sum(len(doc) for doc in raw_documents):,}")
else:
    print("⚠️ Sample data file not found. Using empty list.")
    raw_documents = []


In [ ]:
# Clean and preprocess documents
cleaned_documents = []
for doc in raw_documents:
    cleaned = clean_text(doc)
    cleaned_documents.append(cleaned)

print(f"✅ Cleaned {len(cleaned_documents)} document(s)")
print(f"\nSample cleaned text (first 500 chars):")
if cleaned_documents:
    print(cleaned_documents[0][:500] + "...")


In [ ]:
# Structure documents with metadata
structured_docs = structure_documents(cleaned_documents)

print(f"✅ Structured {len(structured_docs)} document(s)")
print(f"\nSample document structure:")
if structured_docs:
    sample = structured_docs[0]
    print(f"ID: {sample['id']}")
    print(f"Title: {sample.get('title', 'N/A')}")
    print(f"Source: {sample.get('source', 'N/A')}")
    print(f"Text length: {len(sample['text'])} characters")


## ✂️ Section 3: Text Chunking & Preprocessing

### Why Chunk Size Affects Retrieval Accuracy

**Chunk Size Trade-offs:**
- **Too Small**: Loses context, fragments information
- **Too Large**: Includes irrelevant information, reduces precision
- **Optimal**: 500-1000 characters balances context and precision

**Overlap Importance:**
- Prevents information loss at chunk boundaries
- Ensures continuity for multi-chunk answers
- Typical overlap: 10-20% of chunk size


In [ ]:
# Initialize chunker
chunk_size = config.get('chunking', {}).get('chunk_size', 1000)
chunk_overlap = config.get('chunking', {}).get('chunk_overlap', 200)

chunker = MedicalChunker(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    min_chunk_size=100
)

print(f"✅ Chunker initialized:")
print(f"  - Chunk size: {chunk_size} characters")
print(f"  - Overlap: {chunk_overlap} characters")


In [ ]:
# Chunk all documents
all_chunks = []

for doc in structured_docs:
    # Extract metadata
    enhanced_metadata = extract_metadata(doc['text'], doc.get('metadata', {}))
    
    # Chunk document
    chunks = chunker.chunk_text(
        doc['text'],
        document_id=doc['id'],
        metadata=enhanced_metadata
    )
    
    # Convert to dict format
    for chunk in chunks:
        all_chunks.append({
            'text': chunk.text,
            'chunk_id': chunk.chunk_id,
            'document_id': chunk.document_id,
            'metadata': chunk.metadata
        })

print(f"✅ Created {len(all_chunks)} chunks from {len(structured_docs)} documents")
print(f"\nChunk statistics:")
chunk_lengths = [len(c['text']) for c in all_chunks]
print(f"  - Average length: {np.mean(chunk_lengths):.0f} characters")
print(f"  - Min length: {min(chunk_lengths)} characters")
print(f"  - Max length: {max(chunk_lengths)} characters")


## 🔢 Section 4: Embedding & Vector Store

### How Embeddings Represent Medical Concepts

**Embeddings** convert text into numerical vectors that capture semantic meaning:
- Similar medical concepts have similar vectors
- Enables mathematical similarity search
- Preserves relationships between terms

**Why Vector Search is Required:**
- **Speed**: Much faster than keyword search
- **Semantic Understanding**: Finds conceptually similar content
- **Scalability**: Handles large knowledge bases efficiently


In [ ]:
# Initialize embedding generator
model_name = config.get('embedding', {}).get('model_name', 'sentence-transformers/all-MiniLM-L6-v2')
batch_size = config.get('embedding', {}).get('batch_size', 32)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

embedding_generator = EmbeddingGenerator(
    model_name=model_name,
    device=device
)

print(f"✅ Embedding generator initialized")
print(f"  - Model: {model_name}")
print(f"  - Embedding dimension: {embedding_generator.embedding_dim}")


In [ ]:
# Generate embeddings for all chunks
chunk_texts = [chunk['text'] for chunk in all_chunks]

print(f"Generating embeddings for {len(chunk_texts)} chunks...")
embeddings = embedding_generator.generate_embeddings(
    chunk_texts,
    batch_size=batch_size,
    show_progress=True
)

print(f"\n✅ Generated embeddings")
print(f"  - Shape: {embeddings.shape}")
print(f"  - Data type: {embeddings.dtype}")


In [ ]:
# Build vector store
vector_store_path = Path("data/vector_store")

vector_store = build_vector_store(
    embeddings=embeddings,
    chunks=all_chunks,
    save_path=vector_store_path
)

print(f"✅ Vector store built and saved")
print(f"  - Total vectors: {vector_store.get_stats()['total_vectors']}")
print(f"  - Saved to: {vector_store_path}")


## 🔍 Section 5: Retrieval Pipeline

### Why Retrieval Reduces Hallucinations

**Without Retrieval (LLM-only):**
- LLM relies on training data (may be outdated)
- Can generate plausible but incorrect information
- No way to verify sources

**With Retrieval (RAG):**
- Answers grounded in actual documents
- Can cite specific sources
- Knowledge base can be updated independently
- Reduces fabrication of medical facts


In [ ]:
# Initialize retrieval engine
top_k = config.get('retrieval', {}).get('top_k', 5)

retrieval_engine = RetrievalEngine(
    vector_store=vector_store,
    embedding_generator=embedding_generator,
    top_k=top_k,
    score_threshold=0.0
)

print(f"✅ Retrieval engine initialized")
print(f"  - Top-K: {top_k}")


## 🤖 Section 6: LLM Answer Generation (RAG)

### How the LLM Uses Context from Guidelines

**Prompt Construction:**
1. Retrieved documents are formatted as context
2. User query is added
3. Instructions guide the LLM to use only the context
4. Citation requirements are specified

**Why Prompting is Critical:**
- Prevents LLM from using outdated training data
- Ensures answers are evidence-based
- Enforces citation requirements
- Maintains medical safety standards


In [ ]:
# Initialize RAG pipeline
llm_model_name = config.get('llm', {}).get('model_name', 'google/flan-t5-base')
use_api = config.get('llm', {}).get('use_api', False)
api_key = config.get('llm', {}).get('api_key')

print(f"Initializing RAG Pipeline...")
print(f"  - LLM Model: {llm_model_name}")
print(f"  - Use API: {use_api}")

rag_pipeline = RAGPipeline(
    retrieval_engine=retrieval_engine,
    llm_model_name=llm_model_name,
    use_api=use_api,
    api_key=api_key
)

print(f"\n✅ RAG Pipeline initialized")


In [ ]:
# Test the complete RAG pipeline
test_query = "What are the symptoms of malaria?"

print(f"Query: {test_query}\n")
print("Generating answer...\n")

result = rag_pipeline.generate_answer(
    query=test_query,
    top_k=5,
    include_citations=True
)

print("=" * 80)
print("ANSWER:")
print("=" * 80)
print(result['answer'])
print("\n" + "=" * 80)
print(f"\nSources: {len(result['sources'])}")
print(f"Valid: {result['is_valid']}")


## 🧪 Section 7: Testing & Evaluation

### Real-World Performance Demonstration

This section demonstrates:
- Answer quality on medical questions
- Citation accuracy
- Comparison with baseline (LLM-only)
- Evaluation metrics


In [ ]:
# Load test questions
test_questions_file = Path("data/test_questions.json")

if test_questions_file.exists():
    with open(test_questions_file, 'r') as f:
        test_questions = json.load(f)
    print(f"✅ Loaded {len(test_questions)} test questions")
else:
    print("⚠️ Test questions file not found. Using sample questions.")
    test_questions = [
        {"query": "What are the symptoms of malaria?"},
        {"query": "How is tuberculosis treated?"}
    ]


In [ ]:
# Test on multiple questions
print("Testing RAG System on Medical Questions:\n")
print("=" * 80)

for i, test_case in enumerate(test_questions[:3], 1):  # Test first 3
    query = test_case['query']
    print(f"\n[{i}] Query: {query}")
    print("-" * 80)
    
    result = rag_pipeline.generate_answer(query, top_k=3)
    
    print(f"Answer: {result['answer'][:300]}...")
    print(f"\nSources: {len(result['sources'])}")
    if result.get('safety_warning'):
        print(f"Safety Warning: {result['safety_warning']}")
    print("=" * 80)


## 🚀 Section 8: Making it App-Ready (Bonus)

### Deployment Possibilities

This notebook structure can easily be converted into:

1. **FastAPI Endpoint**: REST API for medical Q&A
2. **Streamlit App**: Interactive web interface
3. **Chatbot Interface**: Conversational medical assistant
4. **Mobile App Backend**: API for mobile health apps

### Next Steps for Production

1. **Add Authentication**: Secure API access
2. **Rate Limiting**: Prevent abuse
3. **Logging**: Track usage and errors
4. **Monitoring**: Health checks and metrics
5. **Caching**: Cache common queries
6. **Database**: Store query history
7. **CI/CD**: Automated testing and deployment


## 📊 Summary & Next Steps

### What We Built

✅ **Complete RAG Pipeline**: From data ingestion to answer generation
✅ **Medical Domain Focus**: Safety checks and evidence-based answers
✅ **Production-Ready Code**: Modular, documented, and scalable
✅ **Evaluation Framework**: Metrics and benchmarking tools

### Key Learnings

1. **RAG Architecture**: How retrieval enhances LLM responses
2. **Medical Safety**: Importance of validation and disclaimers
3. **Vector Search**: Efficient semantic similarity search
4. **Prompt Engineering**: Critical for medical accuracy

### Future Enhancements

- **Larger Knowledge Base**: Add more medical guidelines
- **Better Embeddings**: Fine-tune on medical text
- **Reranking**: Improve retrieval quality
- **Multi-turn Conversations**: Context-aware follow-ups
- **Multilingual Support**: Answer in multiple languages

---

**Thank you for using the Medical RAG Q&A Agent!**

Remember: This system is for educational purposes only. Always consult healthcare professionals for medical advice.
